# Background

Objectives:
- To answer the question "Decide between a prime-location HDB or a more remote HDB"

# Import Libraries

In [3]:
import os
os.chdir("../")

In [4]:
import pandas as pd

# Import Data

In [5]:
df = pd.read_parquet('data/raw/resale_hdb_data.parquet')

In [6]:
df["resale_price"] = df["resale_price"].astype(float)
df["floor_area_sqft"] = df["floor_area_sqm"].astype(float) * 10.7639
df["price_per_sqft"] = df["resale_price"] / df["floor_area_sqft"]
df["year"] = df["month"].apply(lambda x: x[:4])

In [7]:
df.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,floor_area_sqft,price_per_sqft,year
0,1,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69,Improved,1986,147000.0,None,742.7091,197.924059,2000
1,2,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61,Improved,1986,144000.0,None,656.5979,219.312307,2000
2,3,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73,New Generation,1976,159000.0,None,785.7647,202.350653,2000
3,4,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73,New Generation,1976,167000.0,None,785.7647,212.531818,2000
4,5,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67,New Generation,1976,163000.0,None,721.1813,226.018062,2000


# Preliminary Analysis

In [88]:
prime_location_towns = ["BUKIT MERAH", "CENTRAL AREA", "QUEENSTOWN", "KALLANG/WHAMPOA"]
fourrm_prime_2023_mask = (
    (df["year"] == "2023") & (df["town"].isin(prime_location_towns)) & (df["flat_type"] == "4 ROOM")
)
df_temp = (
    df[fourrm_prime_2023_mask]
    .groupby(["town"])
    .agg({"_id": "count", "resale_price": ["mean", "median"], "price_per_sqft": ["mean"]})
    .reset_index() \
    .sort_values(by=[("price_per_sqft", "mean")], ascending=False)
)

df_temp["price_per_sqft"] = df_temp["price_per_sqft"].round()
df_temp.columns = ["town", "count", "avg_price", "median_price", "avg_price_per_sqft"]

In [89]:
print(df_temp.to_markdown(index=False))

| town            |   count |   avg_price |   median_price |   avg_price_per_sqft |
|:----------------|--------:|------------:|---------------:|---------------------:|
| CENTRAL AREA    |      77 |      913977 |         800000 |                  929 |
| QUEENSTOWN      |     261 |      859895 |         878000 |                  880 |
| BUKIT MERAH     |     428 |      803362 |         845750 |                  803 |
| KALLANG/WHAMPOA |     294 |      759759 |         780000 |                  757 |


In [39]:
# Percentage of 4-room transactions in prime locations vs all locationsc
fourrm_all_2023_mask = (df["year"] == "2023") & (df["flat_type"] == "4 ROOM")
len(df[fourrm_prime_2023_mask]) / len(df[fourrm_all_2023_mask])

0.09327701513551566

In [92]:
fourrm_2023_mask = (df["year"] == "2023") & (df["flat_type"] == "4 ROOM")
df_temp = df[fourrm_all_2023_mask] \
    .groupby(["town"]) \
    .agg({"_id": "count", "resale_price": ["mean", "median"], "price_per_sqft": ["mean"]}) \
    .reset_index() \
    .sort_values(by=[("price_per_sqft", "mean")], ascending=True)

# round the price per sqft to whole number
df_temp["price_per_sqft"] = df_temp["price_per_sqft"].round()
df_temp.columns = ["town", "count", "avg_price", "median_price", "avg_price_per_sqft"]
print(df_temp.to_markdown(index=False))

| town            |   count |   avg_price |   median_price |   avg_price_per_sqft |
|:----------------|--------:|------------:|---------------:|---------------------:|
| CHOA CHU KANG   |     596 |      502280 |         500000 |                  471 |
| JURONG EAST     |     142 |      490796 |         477500 |                  476 |
| JURONG WEST     |     597 |      499462 |         486000 |                  480 |
| WOODLANDS       |     958 |      499028 |         495000 |                  484 |
| YISHUN          |     870 |      497659 |         498000 |                  496 |
| BUKIT PANJANG   |     367 |      515237 |         498000 |                  502 |
| PASIR RIS       |     269 |      566849 |         548000 |                  515 |
| HOUGANG         |     682 |      546504 |         535000 |                  529 |
| TAMPINES        |     674 |      577373 |         565000 |                  543 |
| SEMBAWANG       |     528 |      545296 |         552500 |                

# Calculate Annualized Growth Rate

In [8]:
import pandas as pd


def calculate_yoy_increase(df):
    """
    Calculate the Year-on-Year (YoY) increase in average resale prices for '4 ROOM' HDB flats
    in prime and remote locations.

    Args:
    df (pd.DataFrame): The DataFrame containing HDB resale data.

    Returns:
    pd.DataFrame: A DataFrame containing the YoY increase and annualized increase in average resale prices.
    """
    # Define prime and remote towns
    prime_towns = ["BUKIT MERAH", "CENTRAL AREA", "QUEENSTOWN", "KALLANG/WHAMPOA"]
    remote_towns = ["CHOA CHU KANG", "JURONG EAST", "JURONG WEST", "WOODLANDS", "YISHUN", "BUKIT PANJANG", "PASIR RIS"]

    # Filter for '4 ROOM' flats
    mask = (df["flat_type"] == "4 ROOM") & (df["year"] >= "2003") & (df["year"] <= "2023")
    df_4_room = df[mask]

    # Split data into prime and remote
    prime_df = df_4_room[df_4_room["town"].isin(prime_towns)]
    remote_df = df_4_room[df_4_room["town"].isin(remote_towns)]

    # Calculate yearly average resale price
    prime_yearly_avg = prime_df.groupby("year")["price_per_sqft"].mean()
    remote_yearly_avg = remote_df.groupby("year")["price_per_sqft"].mean()

    # Calculate YoY increase
    prime_yoy = prime_yearly_avg.pct_change().fillna(0) * 100
    remote_yoy = remote_yearly_avg.pct_change().fillna(0) * 100

    # Calculate annualized increase over 20 years
    prime_annualized = ((prime_yearly_avg.iloc[-1] / prime_yearly_avg.iloc[0]) ** (1 / 20) - 1) * 100
    remote_annualized = ((remote_yearly_avg.iloc[-1] / remote_yearly_avg.iloc[0]) ** (1 / 20) - 1) * 100

    # Print annualized increases
    print(f"Prime Annualized Increase: {prime_annualized:.2f}%")
    print(f"Remote Annualized Increase: {remote_annualized:.2f}%")

    # Prepare final DataFrame
    result_df = pd.DataFrame(
        {
            "Year": prime_yoy.index,
            "Prime PSF YoY %": prime_yoy.values.round(2),
            "Remote PSF YoY %": remote_yoy.values.round(2),
        }
    )

    return result_df, prime_annualized, remote_annualized


df_temp, prime_annualized, remote_annualized = calculate_yoy_increase(df)

Prime Annualized Increase: 5.42%
Remote Annualized Increase: 4.59%


In [9]:
print(df_temp.to_markdown(index=False))


|   Year |   Prime PSF YoY % |   Remote PSF YoY % |
|-------:|------------------:|-------------------:|
|   2003 |              0    |               0    |
|   2004 |              1.26 |               6.1  |
|   2005 |             -0.07 |              -1.69 |
|   2006 |              4.36 |              -1.13 |
|   2007 |             17.89 |               5.58 |
|   2008 |             19.47 |              19.53 |
|   2009 |              5.06 |               9.72 |
|   2010 |             12.86 |              12.7  |
|   2011 |             11.11 |              13.75 |
|   2012 |             10.06 |               8.1  |
|   2013 |              4.96 |               3.15 |
|   2014 |             -2.93 |              -7.34 |
|   2015 |              3.95 |              -6.18 |
|   2016 |             -0.76 |               0.14 |
|   2017 |              0.02 |              -1.03 |
|   2018 |              1.19 |              -2.89 |
|   2019 |             -0.77 |               3.38 |
|   2020 |  

In [12]:
import plotly.graph_objects as go

# Add your calculated annualized increases here
prime_annualized_increase = prime_annualized
remote_annualized_increase = remote_annualized

# Create a line chart
fig = go.Figure()

# Add traces for prime and remote YoY increase
fig.add_trace(
    go.Scatter(x=df_temp["Year"], y=df_temp["Prime PSF YoY %"], mode="lines", name="Prime YoY Increase (%)")
)
fig.add_trace(
    go.Scatter(
        x=df_temp["Year"], y=df_temp["Remote PSF YoY %"], mode="lines", name="Remote YoY Increase (%)"
    )
)

# Update layout for a cleaner look
fig.update_layout(
    title="<b>YoY Change in Average PSF for 4-Room Resale HDBs</b>",
    xaxis_title="Year",
    yaxis_title="YoY Increase (%)",
    template="plotly_white",
    font=dict(family="Arial, sans-serif", size=12, color="#000"),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
)

# Show the plot
fig.show()

In [13]:

import plotly.offline as pyo

# Convert the figure to an HTML div string
div = pyo.plot(fig, output_type='div', include_plotlyjs='cdn')

with open('plot.html', 'w') as file:
    file.write(div)

# Calculate Interest Expense and Future Value

In [12]:
def calculate_interest_expense(
    loan_amount: float, interest_rate: float, tenure_years: int, number_of_years: int
) -> float:
    """
    Calculate the total interest expense payable for a specified number of years of a loan.

    This function computes the interest expense for the first 'number_of_years' of an amortizing loan
    based on the loan amount, annual interest rate, and total tenure of the loan in years.
    'number_of_years' must be less than or equal to 'tenure_years'.

    Args:
        loan_amount (float): The principal amount of the loan.
        interest_rate (float): The annual interest rate as a percentage (e.g., 4 for 4%).
        tenure_years (int): The total tenure of the loan in years.
        number_of_years (int): The number of years for which the interest is calculated.

    Returns:
        float: The interest expense payable over the specified number of years.

    Raises:
        ValueError: If 'number_of_years' is greater than 'tenure_years'.

    Example:
        >>> calculate_interest_expense(100000, 4, 15, 10)
        37724.17  # This is an example output and will vary based on the inputs.
    """
    if number_of_years > tenure_years:
        raise ValueError("'number_of_years' must be less than or equal to 'tenure_years'")

    monthly_interest_rate = interest_rate / 12 / 100
    number_of_payments = tenure_years * 12
    payments_for_specified_years = number_of_years * 12

    # Monthly payment calculation using the formula for an amortizing loan
    monthly_payment = loan_amount * monthly_interest_rate / (1 - (1 + monthly_interest_rate) ** -number_of_payments)

    # Total payment for specified years
    total_payment_for_years = monthly_payment * payments_for_specified_years

    # Principal paid till specified years
    principal_paid = loan_amount
    for _ in range(payments_for_specified_years):
        interest_for_month = principal_paid * monthly_interest_rate
        principal_paid -= monthly_payment - interest_for_month

    # Interest for specified years
    interest_for_years = total_payment_for_years - (loan_amount - principal_paid)

    return round(interest_for_years)

In [15]:
calculate_interest_expense(800000, 4, 30, 10)
calculate_interest_expense(375000, 4, 30, 10)


135277

In [22]:
def calculate_future_value(original_price: float, number_of_years: int, annualized_return_rate: float) -> float:
    """
    Calculate the future value of a property based on its original price, number of years, and annualized return rate.

    This function uses the compound interest formula to compute the future value of a property,
    considering a constant annualized return rate over the specified number of years.

    Args:
        original_price (float): The original purchase price of the property.
        number_of_years (int): The number of years over which the property appreciates.
        annualized_return_rate (float): The annualized return rate as a decimal (e.g., 0.05 for 5%).

    Returns:
        float: The future value of the property after the specified number of years.

    Example:
        >>> calculate_future_value(500000, 10, 0.03)
        671415.47  # This is an example output and will vary based on the inputs.
    """
    future_value = original_price * ((1 + annualized_return_rate) ** number_of_years)
    return round(future_value)


In [26]:
# Example usage
print(calculate_future_value(800000, 10, 0.012))
print(calculate_future_value(500000, 10, 0.0093))

901353
548495
